In [161]:
!pip install torchgeometry
!pip install segmentation-models-pytorch

In [ ]:
import os
import pandas as pd
import numpy as np
import cv2
from torchvision.io import read_image
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from tqdm import tqdm
import torchvision.transforms as T
import torchvision.transforms.functional as F
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
import segmentation_models_pytorch as smp
import wandb
from torchgeometry.losses import DiceLoss

In [163]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Cutomize the dataset class

In [ ]:
#Segmentation Dataset for 
class SegmentationDataset(Dataset):
    def __init__(self, img_dir, label_dir, resize=None, transform=None):
        self.img_dir = img_dir
        self.label_dir = label_dir
        self.resize = resize
        self.transform = transform
        self.images = os.listdir(self.img_dir)

    def __len__(self):
        return len(self.images)
    
    def process_mask(self, mask_path):
        image = cv2.imread(mask_path)
        if self.resize:
            image = cv2.resize(image, self.resize)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        lower_red1 = np.array([0, 100, 20])
        upper_red1 = np.array([10, 255, 255])
        lower_red2 = np.array([160,100,20])
        upper_red2 = np.array([179,255,255])
        
        lower_mask_red = cv2.inRange(image, lower_red1, upper_red1)
        upper_mask_red = cv2.inRange(image, lower_red2, upper_red2)
        
        red_mask = lower_mask_red + upper_mask_red
        red_mask[red_mask != 0] = 1

        green_mask = cv2.inRange(image, (36, 25, 25), (70, 255, 255))
        green_mask[green_mask != 0] = 2

        full_mask = cv2.bitwise_or(red_mask, green_mask)
        full_mask = np.expand_dims(full_mask, axis=-1) 
        full_mask = full_mask.astype(np.uint8)
        return full_mask

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        label_path = os.path.join(self.label_dir, self.images[idx])
        image = cv2.imread(img_path)  #  BGR
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert to RGB
        label = self.process_mask(label_path)  
        image = cv2.resize(image, self.resize)
        if self.transform:
            image = self.transform(image)
        return image, label
    
# Customize dataset for training phase and inference
class CustomDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __getitem__(self, index):
        image = self.data[index]
        label = self.targets[index]
        assert image.shape[:2] == label.shape[:2]
        if self.transform:
            transformed = self.transform(image=image, mask=label)
            image = transformed['image'].float()
            label = transformed['mask'].float()
            label = label.permute(2, 0, 1)
        return image, label
    
    def __len__(self):
        return len(self.data)

Load training data

In [165]:
images_path = "/kaggle/input/bkai-igh-neopolyp/train/train/"
image_path = []
mask_path = []
TRAIN_DIR = '/kaggle/input/bkai-igh-neopolyp/train/train'
TRAIN_MASK_DIR = '/kaggle/input/bkai-igh-neopolyp/train_gt/train_gt'
for root, dirs, files in os.walk(TRAIN_DIR):
    for file in files:
        path = os.path.join(root,file)
        image_path.append(path)
for root, dirs, files in os.walk(TRAIN_MASK_DIR):
    for file in files:
        path = os.path.join(root,file)
        mask_path.append(path)
print(len(image_path), len(mask_path))

1000 1000


In [ ]:
# load pretrained model in dataset imagenet with encoder "efficientnet-b7"
model = smp.Unet(
    encoder_name="efficientnet-b7",        
    encoder_weights="imagenet",     
    in_channels=3,                  
    classes=3     
)

In [ ]:
# use this data augmentation
train_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.RandomCrop(height=256, width=256, p=0.5),
    A.GaussNoise(p=0.2),
    A.Rotate(limit=30, p=0.3),
    A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])
val_transforms = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406),std=(0.229, 0.224, 0.225)),
    ToTensorV2(),
])

In [168]:
dataset = SegmentationDataset(img_dir= TRAIN_DIR,
                             label_dir= TRAIN_MASK_DIR,
                             resize= (256,256),
                             transform = None)

In [169]:
batch_size = 8
images_data = []
labels_data = []
for x,y in dataset:
    images_data.append(x)
    labels_data.append(y)

In [170]:
train_size = int(0.8 * len(images_data))
val_size = len(images_data) - train_size

train_dataset = CustomDataset(images_data[:train_size], labels_data[:train_size], transform=train_transforms)
val_dataset = CustomDataset(images_data[train_size:], labels_data[train_size:], transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [171]:
learning_rate = 0.0001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [172]:
color_dict= {0: (0, 0, 0),
             1: (255, 0, 0),
             2: (0, 255, 0)}
def mask_to_rgb(mask, color_dict):
    output = np.zeros((mask.shape[0], mask.shape[1], 3))

    for k in color_dict.keys():
        output[mask==k] = color_dict[k]

    return np.uint8(output)    

In [ ]:
wandb.login(
    key = "d0ecf10144842718b6ac6ce6f532824de05f1819",
)
wandb.init(
    project = "BKAI-IGH NeoPolyp Deep Learning"
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Train_loss,█▄▃▂▂▂▁▁▁▁▁▁▁▁
Val_loss,█▄▃▃▂▂▁▁▂▁▁▁▁▁
Train_loss,0.03875
Val_loss,0.07598


Train the dataset

In [ ]:
num_epochs = 30
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model.to(device)
criterion1 = nn.CrossEntropyLoss()
criterion2 = DiceLoss()
best_val_loss = 999
epoch_bar = tqdm(total=num_epochs, desc='Total Progress')

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        labels = labels.squeeze(dim=1).long()
        outputs = model(images)
    
        loss1 = criterion1(outputs, labels)
        loss2 = criterion2(outputs, labels)
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss.item()
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)
            labels = labels.squeeze(dim=1).long()
            
            outputs = model(images)

            val_loss1 = criterion1(outputs,labels)
            val_loss2 = criterion2(outputs,labels)
            val_loss += val_loss1 + val_loss2

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader):.10f} , Val Loss: {val_loss/len(val_loader):.10f}")
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        checkpoint = { 
            'epoch': epoch,
            'model': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'loss': val_loss,
        }
        save_path = f'model.pth'
        torch.save(checkpoint, save_path)
        
    epoch_bar.update(1)
    wandb.log({'Val_loss': val_loss/len(val_loader),'Train_loss': train_loss/len(train_loader)})
epoch_bar.close()

Total Progress:   0%|          | 0/30 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/wandb/sdk/wandb_run.py:2388: UserWarning: Run (eb6lwxhr) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
Total Progress:   7%|▋         | 2/30 [02:25<34:02, 72.94s/it]


Epoch [1/30], Train Loss: 0.0358490890 , Val Loss: 0.0809399113


Total Progress:   7%|▋         | 2/30 [02:06<29:19, 62.83s/it]

Epoch [2/30], Train Loss: 0.0353193205 , Val Loss: 0.0814467520
Epoch [3/30], Train Loss: 0.0325924499 , Val Loss: 0.0725141168


Total Progress:  10%|█         | 3/30 [03:09<28:30, 63.35s/it]

...

Epoch [28/30], Train Loss: 0.0151742286 , Val Loss: 0.0784072354


Total Progress:  97%|█████████▋| 29/30 [30:14<01:02, 62.39s/it]

Epoch [29/30], Train Loss: 0.0149066471 , Val Loss: 0.0712109357


Total Progress: 100%|██████████| 30/30 [31:16<00:00, 62.56s/it]

Epoch [30/30], Train Loss: 0.0146322355 , Val Loss: 0.0746502057


In [182]:
checkpoint = torch.load('model.pth')
model.load_state_dict(checkpoint['model'])
device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
model.to(device)

/tmp/ipykernel_30/4249454291.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('model.pth')


Unet(
  (encoder): EfficientNetEncoder(
    (_conv_stem): Conv2dStaticSamePadding(
      3, 64, kernel_size=(3, 3), stride=(2, 2), bias=False
      (static_padding): ZeroPad2d((0, 1, 0, 1))
    )
    (_bn0): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_blocks): ModuleList(
      (0): MBConvBlock(
        (_depthwise_conv): Conv2dStaticSamePadding(
          64, 64, kernel_size=(3, 3), stride=[1, 1], groups=64, bias=False
          (static_padding): ZeroPad2d((1, 1, 1, 1))
        )
        (_bn1): BatchNorm2d(64, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
        (_se_reduce): Conv2dStaticSamePadding(
          64, 16, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_se_expand): Conv2dStaticSamePadding(
          16, 64, kernel_size=(1, 1), stride=(1, 1)
          (static_padding): Identity()
        )
        (_project_conv): Conv2dStaticSamePaddi

In [183]:
!mkdir prediction

In [184]:
model.eval()
for i in os.listdir("/kaggle/input/bkai-igh-neopolyp/test/test"):
    img_path = os.path.join("/kaggle/input/bkai-igh-neopolyp/test/test", i)
    ori_img = cv2.imread(img_path)
    ori_img = cv2.cvtColor(ori_img, cv2.COLOR_BGR2RGB)
    ori_w = ori_img.shape[0]
    ori_h = ori_img.shape[1]
    img = cv2.resize(ori_img, (256, 256))
    transformed = val_transforms(image=img)
    input_img = transformed["image"]
    input_img = input_img.unsqueeze(0).to(device)
    with torch.no_grad():
        output_mask = model.forward(input_img).squeeze(0).cpu().numpy().transpose(1,2,0)
    mask = cv2.resize(output_mask, (ori_h, ori_w))
    mask = np.argmax(mask, axis=2)
    mask_rgb = mask_to_rgb(mask, color_dict)
    mask_rgb = cv2.cvtColor(mask_rgb, cv2.COLOR_RGB2BGR)
    cv2.imwrite("/kaggle/working/prediction/{}".format(i), mask_rgb) 

In [185]:
def rle_to_string(runs):
    return ' '.join(str(x) for x in runs)

def rle_encode_one_mask(mask):
    pixels = mask.flatten()
    pixels[pixels > 225] = 255
    pixels[pixels <= 225] = 0
    use_padding = False
    if pixels[0] or pixels[-1]:
        use_padding = True
        pixel_padded = np.zeros([len(pixels) + 2], dtype=pixels.dtype)
        pixel_padded[1:-1] = pixels
        pixels = pixel_padded
    rle = np.where(pixels[1:] != pixels[:-1])[0] + 2
    if use_padding:
        rle = rle - 1
    rle[1::2] = rle[1::2] - rle[:-1:2]
    
    return rle_to_string(rle)

def rle2mask(mask_rle, shape=(3,3)):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def mask2string(dir):
    strings = []
    ids = []
    ws, hs = [[] for i in range(2)]
    for image_id in os.listdir(dir):
        id = image_id.split('.')[0]
        path = os.path.join(dir, image_id)
        print(path)
        img = cv2.imread(path)[:,:,::-1]
        h, w = img.shape[0], img.shape[1]
        for channel in range(2):
            ws.append(w)
            hs.append(h)
            ids.append(f'{id}_{channel}')
            string = rle_encode_one_mask(img[:,:,channel])
            strings.append(string)
    r = {
        'ids': ids,
        'strings': strings,
    }
    return r


MASK_DIR_PATH = 'prediction/'
dir = MASK_DIR_PATH
res = mask2string(dir)
df = pd.DataFrame(columns=['Id', 'Expected'])
df['Id'] = res['ids']
df['Expected'] = res['strings']

df.to_csv(r'output.csv', index=False)

prediction/0626ab4ec3d46e602b296cc5cfd263f1.jpeg
prediction/88e16d4ca6160127cd1d5ff99c267599.jpeg
prediction/cc5cfd263f1f90be28799235026b3550.jpeg
prediction/5a51625559c7e610b1531871f2fd85a0.jpeg
prediction/77e004e8bfb905b78a91391adc0bb223.jpeg
prediction/780fd497e1c0e9082ea2c193ac8d551c.jpeg
prediction/6ddca6ee1af35b65bd9ea42cfcfedb5e.jpeg
prediction/019410b1fcf0625f608b4ce97629ab55.jpeg
prediction/30c2f4fc276ed9f178dc2f4af6266509.jpeg
prediction/936de314f2d95e6c487ffa651b477422.jpeg
prediction/05b78a91391adc0bb223c4eaf3372eae.jpeg
prediction/f7fdb2d45b21960c94b0aab4c024a573.jpeg
prediction/5c1346e62522325c1b9c4fc9cbe1eca1.jpeg
prediction/a15fc656702fa602bb3c7abacdbd7e6a.jpeg
prediction/f8e26031fbb5e52c41545ba55aadaa77.jpeg
prediction/1ad4f13ccf1f4b331a412fc44655fb51.jpeg
prediction/5beb48f0be11d0309d1dff09b8405734.jpeg
prediction/c7e610b1531871f2fd85a04faeeb2b53.jpeg
prediction/8eb5a9a8a8d7fcc9df8e5ad89d284483.jpeg
prediction/6ad1468996b4a9ce6d840b53a6558038.jpeg
prediction/8fa862560